In [ ]:
from gensim import corpora
from gensim.models import LdaModel
from gensim import similarities
from gensim.parsing.preprocessing import preprocess_string

import re
import nltk
from nltk.corpus import stopwords
nltk.download('all')
from gensim.corpora.dictionary import Dictionary

import pandas as pd

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [ ]:
def preprocessing(a):
  # get rid of non english character
  only_en_pos_review = re.sub('[^a-zA-z]', ' ', a)
  # change it to lower case
  no_cap_pos_review = only_en_pos_review.lower()
  # tokenizing
  word_tokens = nltk.word_tokenize(no_cap_pos_review)
  # pos tagging
  tokens_pos = nltk.pos_tag(word_tokens)
  # only get NN, VB, JJ (noun, verb, adjective)
  final_words = []
  for word, pos in tokens_pos:
    if  pos == 'NN':
      final_words.append(word)
    elif pos == 'VB':
      final_words.append(word)
    elif pos == 'JJ':
      final_words.append(word)
  # lemmatize
  lemma = nltk.WordNetLemmatizer()
  lemmatized_words = []
  for word in final_words:
    new_word = lemma.lemmatize(word)
    lemmatized_words.append(new_word)
  # stop words
  stopwords_list = stopwords.words(['english', 'spanish'])
  # adding stop words
  add_stopwords_list = ['station', 'seoul', 'yokohama','train', 'line', 'place', 'tokyo', 'newyork',
                        'york', 'time', 'kyoto', 'hankyu', 'osaka', 'good', 'shibuya',
                        'lot', 'many']
  stopwords_list = stopwords_list +  add_stopwords_list

  unique_words = set(lemmatized_words)
  final_words = lemmatized_words

  # get rid of stop words
  for word in unique_words:
      if word in stopwords_list or len(word) < 3: # delete when it is stopwords or words with two letters
          while word in final_words: final_words.remove(word)

  return final_words

In [ ]:
import numpy as np
import pandas as pd

import os

def get_csv_files(folder_path):
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    return csv_files

folder_path = 'data/review'

csv_files = sorted(get_csv_files(folder_path))

In [ ]:
def lda_output (file_path,encoder):

  df = pd.read_csv(file_path,encoding=encoder)
  del df ['Unnamed: 0.1']
  del df ['Unnamed: 0']


  documents = []
  for i in df['review']:
    documents.append(i)

  texts = [preprocessing(sentence) for sentence in documents]
  dictionary = Dictionary(texts)
  dictionary.filter_extremes(keep_n = 2000, no_below =5, no_above= 0.5)

  corpus = [dictionary.doc2bow(text) for text in texts]


  # change the output by chaning the source of LDA model load
  pos_model = LdaModel.load('data/review_tm/topic_model/topic_num_4/pos_topic_4')
  pos_result = pos_model.get_document_topics(corpus)
  neg_model = LdaModel.load('data/review_tm/topic_model/topic_num_4/neg_topic_4')
  neg_result = neg_model.get_document_topics(corpus)
  
  # pos_model = LdaModel.load('data/review_tm/topic_model/topic_num_5/pos_topic_5')
  # pos_result = pos_model.get_document_topics(corpus)
  # neg_model = LdaModel.load('data/review_tm/topic_model/topic_num_5/neg_topic_5')
  # neg_result = neg_model.get_document_topics(corpus)
  
  # pos_model = LdaModel.load('data/review_tm/topic_model/topic_num_6/pos_topic_6')
  # pos_result = pos_model.get_document_topics(corpus)
  # neg_model = LdaModel.load('data/review_tm/topic_model/topic_num_6/neg_topic_6')
  # neg_result = neg_model.get_document_topics(corpus)

  # change the number of elements in the diciontary
  output_frame = {
    'statnNm' : ['' for _ in range(len(corpus))],
    'pos0' : [0.0 for _ in range(len(corpus))],
    'pos1' : [0.0 for _ in range(len(corpus))],
    'pos2' : [0.0 for _ in range(len(corpus))],
    'pos3' : [0.0 for _ in range(len(corpus))],
    # 'pos4' : [0.0 for _ in range(len(corpus))],
    # 'pos5' : [0.0 for _ in range(len(corpus))],
    'neg0' : [0.0 for _ in range(len(corpus))],
    'neg1' : [0.0 for _ in range(len(corpus))],
    'neg2' : [0.0 for _ in range(len(corpus))],
    'neg3' : [0.0 for _ in range(len(corpus))],
    # 'neg4' : [0.0 for _ in range(len(corpus))],
    # 'neg5' : [0.0 for _ in range(len(corpus))],
  }
  output = pd.DataFrame(output_frame)

  name = list(df.statnNm.to_numpy())

  for i in range(len(corpus)):
    output.loc[i,'statnNm'] = name[i]
    for j in pos_result[i]:
      col_name = 'pos'+ str(j[0])
      output.loc[i,col_name] = j[1]
    for k in neg_result[i]:
      col_name = 'neg'+ str(k[0])
      output.loc[i,col_name] = k[1]

  return output

In [ ]:
d_encoding = [5,7,9,13]

path = 'data/lda_result'

for n, file in enumerate(csv_files):
  path = folder_path+'/'+ file
  if n in d_encoding:
    output = lda_output(path,'cp949')
  else:
    output = lda_output(path,'utf-8')
  output_name = file[:-4]+'_lda_result.csv'
  output.to_csv(path+output_name)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
